# **Colab Tutorial: Rewriting Prompts with Gemini API (`gemini-2.5-pro-exp-03-25` - Client Approach - Updated Limits)**

## 1. Introduction

Welcome! This tutorial guides you through using the Google Gemini API (`gemini-2.5-pro-exp-03-25` model) via the `genai.Client` interface to rewrite text prompts. We'll read prompts from an uploaded CSV file, use the Gemini API for rewriting, handle errors, manage rate limits (specifically the **2 RPM / 50 per day free tier limit**), and save the results.

**Focus:** Using the `genai.Client` method, constructing requests, sending them via `client.models.generate_content`, handling specific errors with retries, and managing updated rate limits.

**What this script does:**
1.  Reads prompts from an uploaded CSV file (requires a `prompt` column).
2.  Uses the `genai.Client` to interact with the `gemini-2.5-pro-exp-03-25` model.
3.  Sends requests using `client.models.generate_content` with retry logic.
4.  Prints interaction details (request attempt, response, errors) to the output.
5.  Respects the 2 Requests Per Minute limit with appropriate delays.
6.  Saves the original data along with rewritten prompts to an output CSV.

## 2. Prerequisites

* **Google Account:** To use Google Colab.
* **Gemini API Key:** Get one from [Google AI Studio](https://aistudio.google.com/app/apikey).
    * **Note:** Access to `gemini-2.5-pro-exp-03-25` might be restricted. If API calls fail with model errors, check your access.
* **An Input CSV File:** You need a CSV file ready to upload. This file **must contain a column named `prompt`**. It can contain other columns as well, which will be preserved in the output.
* **Basic Python & Pandas Knowledge.**

## 3. Setup: Install Libraries & Configure API Key

Install libraries and securely set up your API Key.

In [ ]:
# @title Install Google Generative AI SDK & Pandas
# We use -q for a quieter installation
!pip install -q google-generativeai pandas
print("✅ Libraries installed.")

In [ ]:
# @title Configure API Key (using Colab Secrets is recommended)
import os
from google.colab import userdata

# --- Use Colab Secrets (Recommended) ---
# 1. Click the key icon (Secrets) in the left sidebar.
# 2. Add a new secret named GOOGLE_API_KEY.
# 3. Paste your API key as the value and enable "Notebook access".
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key loaded from Colab Secrets.")
except userdata.SecretNotFoundError:
    print("🚨 Secret 'GOOGLE_API_KEY' not found. Please create the secret.")
    GOOGLE_API_KEY = None
except Exception as e:
    print(f"🚨 An error occurred accessing secrets: {e}")
    GOOGLE_API_KEY = None

# --- Verification ---
if not GOOGLE_API_KEY:
    print("🚨 API Key is not configured. The script cannot run without it.")
else:
    print("✅ API Key is ready to be used.")

## 4. Configuration and Client Setup

Define configurations, **updated rate limits**, and initialize the `genai.Client`.

In [ ]:
# @title Configuration & Client Initialization
import time
import pandas as pd
from google import genai
from google.api_core.exceptions import ResourceExhausted, InternalServerError

# --- Model & API Configuration ---
API_KEY = GOOGLE_API_KEY # Use the key loaded in the previous step
MODEL_NAME = 'gemini-2.5-pro-exp-03-25'
# Note: Ensure you have access to this specific experimental model.

# --- Initialize the Gemini Client ---
client = None # Initialize client as None
if API_KEY:
    try:
        # *** Using genai.Client as requested ***
        client = genai.Client(api_key=API_KEY)
        print(f"✅ Initialized genai.Client.")
    except Exception as e:
        print(f"🚨 Error initializing genai.Client: {e}")
else:
    print("🚨 Cannot initialize client: API Key not configured.")


# --- Rate Limit Constants (Updated for Free Tier: 2 RPM / 50 per day) ---
RATE_LIMIT_REQUESTS_PER_MINUTE = 2 # Free tier limit based on screenshot
# Calculate delay based on 2 RPM. Add a small buffer (e.g., 1 second) for safety.
# 60 seconds / 2 RPM = 30 seconds/request.
MIN_DELAY_BETWEEN_REQUESTS = (60 / RATE_LIMIT_REQUESTS_PER_MINUTE) + 1 # Add 1s buffer = 31 seconds

# Also, keep in mind the 50 requests/day limit for longer runs!
# The script doesn't automatically track this daily limit.
print(f"Using FREE TIER Rate Limits: ~{RATE_LIMIT_REQUESTS_PER_MINUTE} RPM (min delay: {MIN_DELAY_BETWEEN_REQUESTS:.2f}s). Daily limit: 50 requests.")
print("⚠️ NOTE: The daily limit of 50 requests is NOT tracked by this script. Monitor usage for long runs.")

✅ Initialized genai.Client.
Using FREE TIER Rate Limits: ~2 RPM (min delay: 31.00s). Daily limit: 50 requests.
⚠️ NOTE: The daily limit of 50 requests is NOT tracked by this script. Monitor usage for long runs.


## 5. Handling API Requests with Retries

Using the `send_request_with_retry` function you provided (fixed delay, raises Exception). This function will now be the primary place where model name errors are caught during execution.

In [ ]:
# @title Function: Send Request to Gemini API (Client Method, with Retries - As Provided)

def send_request_with_retry(contents, retries=3, delay=10):
    """
    Send a request to Gemini using client.models.generate_content
    with retry logic for handling errors (fixed delay).
    Raises Exception on final failure.

    Args:
        contents (str): The prompt content to send.
        retries (int): Maximum number of retry attempts.
        delay (int): Fixed delay in seconds between retries.

    Returns:
        str: The text response from Gemini.

    Raises:
        Exception: If the request fails after all retries or client not initialized.
    """
    if not client:
        print("--- ERROR: Gemini client not initialized. Cannot send request. ---")
        # Raise an exception immediately if client isn't setup
        raise Exception("Gemini client not initialized.")

    for attempt in range(retries):
        print(f"--- Attempt {attempt + 1}/{retries}: Sending request to Gemini ({MODEL_NAME})... ---")
        try:
            # *** The core API call using client.models.generate_content ***
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=[contents]  # Wrap the string prompt in a list
            )
            # Check if the response has text (basic validation)
            if not hasattr(response, 'text'):
                 print(f"--- WARNING (Attempt {attempt + 1}): Response object lacks 'text' attribute. Response: {response} ---")
                 # Decide how to handle: raise, return marker, or retry? Raising here.
                 raise Exception(f"Unexpected response format (attempt {attempt+1})")

            print(f"--- Attempt {attempt + 1}: Success! Received response. ---")
            return response.text # Return the generated text

        # *** Handling Specific API Errors ***
        except ResourceExhausted:
            # Using fixed delay from function args
            print(f"   API Error: Resource Exhausted (likely rate limit). Retrying in {delay} seconds...")
            time.sleep(delay)

        except InternalServerError as e:
            print(f"   API Error: Internal Server Error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)

        # *** Handling Other Potential Errors (e.g., Model Not Found, Invalid Argument) ***
        except Exception as e:
            print(f"   An unexpected error occurred: {e}. Retrying in {delay} seconds...")
            # Check if error suggests model not found - THIS IS WHERE MODEL ERRORS WILL LIKELY APPEAR
            if "model" in str(e).lower() and ("not found" in str(e).lower() or "invalid" in str(e).lower() or "cannot access" in str(e).lower()):
                 print(f"      ↳ CRITICAL: This error likely indicates the MODEL_NAME ('{MODEL_NAME}') is incorrect or not accessible with your API key.")
            time.sleep(delay)

    # If loop completes without returning, all retries failed
    print(f"--- ERROR: Failed to get response after {retries} retries. Raising exception. ---")
    raise Exception("Failed to send request after retries")

## 6. Preparing the Data and Constructing the Prompt

Reading the CSV and formatting the prompt. `rewrite_prompt` catches exceptions from `send_request_with_retry`.

In [ ]:
# @title Function: Read CSV Data

def read_csv_data(csv_file):
    """Reads data from the CSV file, expecting a 'prompt' column."""
    try:
        df = pd.read_csv(csv_file)
        # *** Crucial Check: Ensure 'prompt' column exists ***
        if 'prompt' not in df.columns:
             print(f"🚨 Error: Uploaded CSV file ('{csv_file}') MUST contain a column named 'prompt'.")
             return None
        print(f"✅ Loaded {len(df)} rows from {csv_file}")
        return df
    except FileNotFoundError:
        print(f"🚨 Error: Input CSV file ('{csv_file}') not found.")
        print("   Please upload the file using the 'Files' tab in the left sidebar before running the script.")
        return None
    except Exception as e:
        print(f"🚨 Error reading CSV file '{csv_file}': {e}")
        return None

# @title Function: Rewrite Prompt (Constructs & Sends Request via Client)

def rewrite_prompt(original_prompt, index):
    """
    Constructs the full prompt for Gemini and uses send_request_with_retry.
    Catches exceptions from the send function. Prints details.

    Args:
        original_prompt (str): The prompt text read from the CSV.
        index (int): The row index (for printing).

    Returns:
        str: The rewritten text from Gemini, or None on failure.
    """
    print(f"\n--- [Row {index+1}] Preparing Prompt ---")
    print(f"Original Prompt:\n{original_prompt}\n")

    # *** This is the instruction template given to the Gemini model ***
    rewriting_instruction = f"""Rewrite the following prompt while keeping all the original information exactly the same:

---
{original_prompt}
---

Guidelines:
- Keep all original information and meaning exactly the same.
- Do not add or remove any details.
- Do not change any requirements or instructions.
- Ensure the output is only the rewritten prompt text, with no introductory phrases like "Here is the rewritten prompt:".

Provide only the rewritten prompt."""

    try:
        # *** Call the API function - this might raise Exception on failure ***
        rewritten_text = send_request_with_retry(rewriting_instruction) # Pass the instruction string

        # If successful, clean and return
        cleaned_text = rewritten_text.strip()
        print(f"--- [Row {index+1}] Received Rewritten Prompt ---")
        print(f"{cleaned_text}\n")
        return cleaned_text

    except Exception as e:
        # Catch the exception raised by send_request_with_retry on final failure
        print(f"--- [Row {index+1}] ERROR: Rewrite failed for this row after retries. Error: {e} ---")
        # The specific error (e.g., model not found) will be printed by send_request_with_retry
        return None # Indicate failure for this row by returning None

## 7. Main Execution Logic

Orchestrates the process: reading, looping, calling rewrite, applying delays, saving. Uses updated `MIN_DELAY_BETWEEN_REQUESTS`.

In [ ]:
# @title Function: Main Orchestrator (Client Method, Updated Limits)

def main(input_csv="data.csv", output_csv="data_rewritten.csv", delay_seconds=None):
    """
    Main function using genai.Client and updated rate limits.
    """
    print("\n--- Starting Prompt Rewriting Script (Client Method - Updated Limits) ---")
    # Ensure client is available before proceeding
    if not client:
        print("🚨 Halting script: Gemini client was not initialized (check API Key and previous cell output).")
        return

    # Determine request delay, ensuring it respects the minimum calculated delay
    if delay_seconds is not None and float(delay_seconds) >= MIN_DELAY_BETWEEN_REQUESTS:
         request_delay = float(delay_seconds)
         print(f"Using custom delay: {request_delay:.2f} seconds.")
    else:
         request_delay = MIN_DELAY_BETWEEN_REQUESTS
         if delay_seconds is not None:
             print(f"Custom delay {delay_seconds}s is less than minimum {MIN_DELAY_BETWEEN_REQUESTS:.2f}s. Using minimum delay.")
         else:
             print(f"Using default minimum delay: {request_delay:.2f} seconds.")

    print(f"Input file: {input_csv}")
    print(f"Output file: {output_csv}")

    # Read data (Checks if file exists and has 'prompt' column)
    df = read_csv_data(input_csv)
    if df is None:
        print("🚨 Exiting: Failed to read input CSV or 'prompt' column missing.")
        return

    # Prepare output DataFrame (Copies all columns from input)
    df_rewritten = df.copy()
    df_rewritten['gemini_rewrite'] = None # Initialize new column

    start_time = time.time()
    processed_count = 0
    error_count = 0
    total_rows = len(df_rewritten)
    print(f"\n--- Starting Processing for {total_rows} prompts ---")
    print(f"⚠️ Daily Limit Reminder: Max 50 requests per day for free tier.")

    # --- Loop through prompts, rewrite, and apply delay ---
    for index, row in df_rewritten.iterrows():
        # Check against daily limit (simple counter)
        if processed_count >= 50:
             print(f"\n--- STOPPING: Reached daily limit approximation (50 requests). ---")
             print(f"   Processed {processed_count} rows. Please run again tomorrow if needed.")
             break # Stop processing more rows

        # Get prompt from the required 'prompt' column
        original_prompt = row['prompt']

        # Basic check for valid prompt data
        if not isinstance(original_prompt, str) or pd.isna(original_prompt):
            print(f"--- [Row {index+1}/{total_rows}] Skipping: Invalid or missing prompt value in 'prompt' column. ---")
            df_rewritten.loc[index, 'gemini_rewrite'] = "[Invalid Input Prompt]"
            continue # Skip to next row

        # Call the rewrite function (handles API call & exceptions)
        rewritten_prompt_text = rewrite_prompt(original_prompt, index) # Pass index for context prints

        processed_count += 1 # Increment count *after* attempting a rewrite API call

        if rewritten_prompt_text is None:
            error_count += 1 # Count errors based on None return from rewrite_prompt

        # Store result (will be None if rewrite_prompt caught an exception)
        df_rewritten.loc[index, 'gemini_rewrite'] = rewritten_prompt_text

        # --- Rate Limiting Delay ---
        # Apply delay *after* each request attempt (except the last one or if daily limit reached)
        if index < total_rows - 1 and processed_count < 50:
            print(f"--- Waiting {request_delay:.2f} seconds before next request... ---")
            time.sleep(request_delay)

    # --- End of Processing ---
    end_time = time.time()
    duration = end_time - start_time
    print(f"\n--- Processing Complete ---")
    print(f"Attempted processing for {processed_count} prompts (out of {total_rows} rows) in {duration:.2f} seconds.")
    if error_count > 0:
        print(f"🚨 Encountered errors during API calls for {error_count} prompts (check output above for details - look for 'ERROR: Rewrite failed').")
    if processed_count >= 50:
        print(f"🛑 Stopped due to reaching the approximate daily limit of 50 requests.")

    # --- Save Results ---
    try:
        # Save the DataFrame (includes original columns + 'gemini_rewrite')
        df_rewritten.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"✅ Successfully saved results to '{output_csv}'")
    except Exception as e:
        print(f"🚨 Error saving output CSV file '{output_csv}': {e}")

    print("--- Script Finished ---")

## 8. Run the Script

**Before running this cell:**
1.  **Upload your input CSV file:** Use the folder icon in the left sidebar and click "Upload to session storage".
2.  Ensure the uploaded file is named `data.csv` (or change `INPUT_CSV_FILE` below).
3.  Ensure the uploaded file contains a column named exactly `prompt`.

Now, execute the `main` function. Monitor the output for API interaction details, delays, and potential errors.

In [ ]:
# @title ▶️ Run the Prompt Rewriting Process (Client Method, Updated Limits)

# Define input and output filenames
# *** Make sure INPUT_CSV_FILE matches the name of your uploaded file ***
INPUT_CSV_FILE = "data.csv"
OUTPUT_CSV_FILE = "data_rewritten.csv"

# Optional: You can try overriding the delay here, but the 'main' function
# will force it to be at least MIN_DELAY_BETWEEN_REQUESTS (e.g., 31s).
CUSTOM_DELAY_SECONDS = None # e.g., 35.0 seconds

# --- Execute the main function ---
try:
    # Ensure all previous function definition cells are run
    main(input_csv=INPUT_CSV_FILE, output_csv=OUTPUT_CSV_FILE, delay_seconds=CUSTOM_DELAY_SECONDS)
except NameError as e:
    print(f"🚨 NameError: {e}. Make sure you ran all function definition cells above first!")
except Exception as e:
    print(f"🚨 An unexpected error occurred during the main execution: {e}")


--- Starting Prompt Rewriting Script (Client Method - Updated Limits) ---
Using default minimum delay: 31.00 seconds.
Input file: data.csv
Output file: data_rewritten.csv
✅ Loaded 83 rows from data.csv

--- Starting Processing for 83 prompts ---
⚠️ Daily Limit Reminder: Max 50 requests per day for free tier.

--- [Row 1] Preparing Prompt ---
Original Prompt:
Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include the word count. Include FAQs section in the outline too, based on people also ask section from Google for the keyword. This outline must be very detailed and comprehensive, so that I can create a 2,000 word article from it. Generate a long list of LSI and NLP keywords related to my keyword. Also include any other words related to the keyword. Give me a list 

KeyboardInterrupt: 

## 9. Check the Output (`data_rewritten.csv`)

After execution finishes or stops:
1.  **Refresh the Colab file browser** (left sidebar folder icon).
2.  Locate the output file (default: `data_rewritten.csv`).
3.  **Inspect the CSV:** Double-click to view or download it. It will contain all columns from your original input file, plus the `gemini_rewrite` column showing the results (or `None` for failures, `[Invalid Input Prompt]` for skipped rows).

## 10. Conclusion & Next Steps

This tutorial demonstrated using the `genai.Client` interface with the specific `gemini-2.5-pro-exp-03-25` model, incorporating updated free tier rate limits (2 RPM / 50 per day).

**Key steps covered:**

* Initializing `genai.Client`.
* Reading prompts from an uploaded CSV (requiring a `prompt` column).
* Using `client.models.generate_content` for API calls.
* Handling errors (including potential model access errors) with fixed-delay retries during the API call.
* Implementing delays based on the 2 RPM limit.
* Adding a basic check for the 50 requests/day limit.

**Next steps:**

* Monitor the output carefully for any API errors, especially those mentioning the model name or access issues during the "send_request_with_retry" attempts.
* If processing more than 50 prompts, plan to run the script over multiple days or adapt it.
* Ensure your input CSV always has the `prompt` column correctly named.

---